# Mixed Layer Ocean Thickness defined by Sigma Theta (MLOTST)

In [1]:
import xarray as xr
from pydap.client import open_url
import numpy as np

In [55]:
chunk_size = {'lat': 177, 'lon': 241, 'time': 500}

In [2]:
mlotst = xr.open_mfdataset('tutorials/motu/cmems_mod_glo_phy_my_0.083_P1D-m/*.nc')

In [3]:
mlotst

<xarray.Dataset>
Dimensions:    (time: 10227, latitude: 529, longitude: 721)
Coordinates:
  * latitude   (latitude) float32 -12.0 -11.92 -11.83 ... 31.83 31.92 32.0
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 2020-12-31T12:00:00
  * longitude  (longitude) float32 42.0 42.08 42.17 42.25 ... 101.8 101.9 102.0
Data variables:
    mlotst     (time, latitude, longitude) float32 dask.array<chunksize=(31, 529, 721), meta=np.ndarray>
Attributes: (12/25)
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2022/05/25 21:54:07 MERCATOR OCEAN Ne...
    source:                             MERCATOR GLORYS12V1
    institution:                        MERCATOR OCEAN
    ...                                 ...
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

In [43]:
ds = xr.open_zarr('shared/data/FINAL_7_19_23.zarr/')

## TIME1

In [13]:
grid_time1 = ds.sel(time=slice('1979', '1992'))['u_wind']

In [14]:
grid_time1

<xarray.DataArray 'u_wind' (time: 5114, lat: 177, lon: 241)>
dask.array<getitem, shape=(5114, 177, 241), dtype=float32, chunksize=(100, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31
Attributes:
    long_name:       10 metre U wind component
    nameCDM:         10_metre_U_wind_component_surface
    nameECMWF:       10 metre U wind component
    product_type:    analysis
    shortNameECMWF:  10u
    standard_name:   eastward_wind
    units:           m s**-1

In [15]:
grid_time1.shape

(5114, 177, 241)

In [16]:
salinity_time1 = grid_time1.copy()

In [17]:
salinity_time2 = salinity_time1.where(~np.isnan(salinity_time1), np.nan)

In [18]:
salinity_time2.values[:] = np.nan

In [19]:
salinity_time2

<xarray.DataArray 'u_wind' (time: 5114, lat: 177, lon: 241)>
dask.array<where, shape=(5114, 177, 241), dtype=float32, chunksize=(100, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31
Attributes:
    long_name:       10 metre U wind component
    nameCDM:         10_metre_U_wind_component_surface
    nameECMWF:       10 metre U wind component
    product_type:    analysis
    shortNameECMWF:  10u
    standard_name:   eastward_wind
    units:           m s**-1

In [20]:
salinity_time2.name = 'mlotst'

In [21]:
salinity_time2.attrs.clear()

In [23]:
chunk_size = {'lat': 177, 'lon': 241, 'time': 100}

In [24]:
salinity_time2 = salinity_time2.chunk(chunk_size)

In [25]:
salinity_time2

<xarray.DataArray 'mlotst' (time: 5114, lat: 177, lon: 241)>
dask.array<where, shape=(5114, 177, 241), dtype=float32, chunksize=(100, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1992-12-31

In [26]:
salinity_time2.to_netcdf('data/mlotst_time1.nc')

## TIME2

In [28]:
grid_time2 = ds.sel(time=slice('1993', '1999'))
salinity_time2 = mlotst.sel(time=slice('1993', '1999'))['mlotst']

In [29]:
salinity_time2 =salinity_time2.rename({'latitude': 'lat', 'longitude': 'lon'})

In [30]:
salinity_time2_interp = salinity_time2.interp_like(grid_time2)

In [31]:
salinity_time2_interp

<xarray.DataArray 'mlotst' (time: 2556, lat: 177, lon: 241)>
dask.array<dask_aware_interpnd, shape=(2556, 177, 241), dtype=float32, chunksize=(2556, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 1993-01-01 1993-01-02 ... 1999-12-31
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
Attributes:
    long_name:      Density ocean mixed layer thickness
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_theta
    units:          m
    unit_long:      Meters
    cell_methods:   area: mean
    _ChunkSizes:    [   1  681 1440]

In [32]:
salinity_time2_interp.attrs.clear()

In [33]:
salinity_time2_interp.to_netcdf('data/mlotst_time2.nc')

## TIME3

In [34]:
grid_time3 = ds.sel(time=slice('2000', '2009'))
salinity_time3 = mlotst.sel(time=slice('2000', '2009'))['mlotst']
salinity_time3 =salinity_time3.rename({'latitude': 'lat', 'longitude': 'lon'})
salinity_time3_interp = salinity_time3.interp_like(grid_time3)
salinity_time3_interp.attrs.clear()

In [35]:
salinity_time3_interp

<xarray.DataArray 'mlotst' (time: 3653, lat: 177, lon: 241)>
dask.array<dask_aware_interpnd, shape=(3653, 177, 241), dtype=float32, chunksize=(3653, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2009-12-31
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0

In [36]:
salinity_time3_interp.to_netcdf('data/mlotst_time3.nc')

## TIME4

In [37]:
grid_time4 = ds.sel(time=slice('2010', '2020'))
salinity_time4 = mlotst.sel(time=slice('2010', '2020'))['mlotst']
salinity_time4 =salinity_time4.rename({'latitude': 'lat', 'longitude': 'lon'})
salinity_time4_interp = salinity_time4.interp_like(grid_time4)
salinity_time4_interp.attrs.clear()

In [38]:
salinity_time4_interp

<xarray.DataArray 'mlotst' (time: 4018, lat: 177, lon: 241)>
dask.array<dask_aware_interpnd, shape=(4018, 177, 241), dtype=float32, chunksize=(4018, 177, 241), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * time     (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2020-12-31
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0

In [39]:
salinity_time4_interp.to_netcdf('data/mlotst_time4.nc')

## TIME5

In [47]:
grid_time5 = ds['u_wind'].sel(time=slice('2021', '2022'))
salinity_time5=grid_time5.copy()
salinity_time5.name='mlotst'
salinity_time5.attrs.clear()

In [48]:
salinity_time5.load()
salinity_time5.values[:] = np.nan

## Combine stuff

In [42]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/15)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ...               ...
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [49]:
salinity_imported = xr.open_mfdataset('data/mlotst_time*.nc')

In [50]:
salinity = xr.combine_by_coords([salinity_imported, salinity_time5])

In [51]:
salinity

<xarray.Dataset>
Dimensions:  (lat: 177, lon: 241, time: 16071)
Coordinates:
  * lat      (lat) float32 32.0 31.75 31.5 31.25 ... -11.25 -11.5 -11.75 -12.0
  * lon      (lon) float32 42.0 42.25 42.5 42.75 ... 101.2 101.5 101.8 102.0
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables:
    mlotst   (time, lat, lon) float32 dask.array<chunksize=(5114, 177, 241), meta=np.ndarray>

In [53]:
salinity['mlotst'].attrs = mlotst['mlotst'].attrs

In [54]:
ds['mlotst'] = (['time', 'lat', 'lon'], salinity['mlotst'].values)

In [57]:
ds = ds.chunk(chunk_size)

In [58]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/17)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    direction        (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    sla              (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ...               ...
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    mlotst           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [59]:
ds['mlotst'].attrs = mlotst['mlotst'].attrs

In [60]:
ds['mlotst'].attrs

{'long_name': 'Density ocean mixed layer thickness',
 'standard_name': 'ocean_mixed_layer_thickness_defined_by_sigma_theta',
 'units': 'm',
 'unit_long': 'Meters',
 'cell_methods': 'area: mean',
 '_ChunkSizes': array([   1,  681, 1440], dtype=int32)}

In [61]:
ds.to_zarr('shared/data/FINAL_7_19_23_2.zarr/', consolidated=True)

In [63]:
ds =  xr.open_zarr('shared/data/FINAL_7_19_23_2.zarr/')

In [64]:
ds = ds.rename({'speed': 'wind_speed', 'direction': 'wind_dir'})

In [65]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/17)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    mlotst           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ...               ...
    u_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    u_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [70]:
ds =ds.assign(curr_speed=np.sqrt(ds.u_curr**2+ds.v_curr**2))

In [71]:
ds =ds.assign(curr_dir=np.rad2deg(np.arctan2(ds.v_curr, ds.u_curr)))

In [72]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/19)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    mlotst           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ...               ...
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [75]:
ds.curr_speed.attrs = {
    'long_name': 'average total surface current speed',
    'units': 'm s**-1',
    'depth': '15m',
    'comments': 'Velocities are an average over the top 30m of the mixed layer'
}

ds.curr_dir.attrs = {
    'long_name': 'average direction of total surface currents',
    'units': 'degrees',
    'depth': '15m',
    'comments': 'Computed from total surface current velocity elements. Velocities are an average over the top 30m of the mixed layer'
}

In [76]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/19)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    mlotst           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ...               ...
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-07
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [77]:
ds.attrs['date_created'] = '2023-07-19'

In [78]:
ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/19)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    mlotst           (time, lat, lon) float32 dask.array<chunksize=(500, 177, 241), meta=np.ndarray>
    ...               ...
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-19
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [79]:
ds.to_zarr('shared/data/INDIAN_OCEAN_025GRID_DAILY.zarr')